In [ ]:
library(Seurat)
library(data.table)
library(ggplot2)
library(readxl)
library(plyr)

library(dplyr)

library(reshape2)

library(viridis)
library(knitr)
library(stringr)
library(NMF)
library(rsvd)
library(lme4)
library(RColorBrewer)

library(gtools)
library(gprofiler2)
library(destiny)
# Plot
library(conflicted)
#suppressPackageStartupMessages(library(scran))
library(purrr)

library(viridis)
library(cowplot)
library(ggplot2)
library(ggbeeswarm)
library(ggthemes)

In [ ]:
setwd(".../SVZ/Test_Code")

NG <- readRDS(".../RData/NSCs_neurons_CCA_scaled.rds")
DefaultAssay(NG) <- "RNA"
levels(NG)

In [ ]:
Idents(NG) <- NG$cluster

order<-c('16','0','7','8', '19',
            '11','13','14',
         '2','3','12','10',
                        '9')



Idents(NG) <- factor(x = Idents(NG), levels = order)
levels(NG)

In [ ]:
matrix<-as.matrix(GetAssayData(NG, assay = "integrated"))

#matrix<-t(matrix)#cells should be rows
var_genes<-rownames(matrix)

matrix <- as.matrix(GetAssayData(NG,slot = "data")[var_genes,])
matrix<-t(matrix)#cells should be rows

In [ ]:
NSC_cells <- names(NG@active.ident)

In [ ]:
sigmas<-find_sigmas(matrix, verbose= FALSE)
optimal<-optimal_sigma(sigmas)

In [ ]:
NG_DiffussionMap <- DiffusionMap(matrix,optimal) 
NG_DPT <- rank(-NG_DiffussionMap$DC1) # We take the 1st DC, DC1 are negative values.. why?


NG_cells <- names(NG@active.ident)
NG_RNA <- data.frame(Cell_type=as.vector(unlist(NG@active.ident[NG_cells])), Landmark=rep('NG', length(NG_cells)), Pseudotime=NG_DPT, PosLandmark=rep('Before', length(NG_cells)),row.names = NG_cells)


In [ ]:
NG_RNA_2<-NG_RNA
NG_RNA_2$Cell_type<-factor(NG_RNA_2$Cell_type, levels=c('16','0','7','8',
                            '19',
                            '11','13','14',
                            '2','3','12','10',
                            '9'))
head(NG_RNA_2)
levels(NG_RNA_2$Cell_type)

In [ ]:
ucols_NG<-c("#993399", "#FFCCFF", '#CC0099' ,
             "#999999",
          
             "#333333",'#3333FF',"#33CCFF","#99FFFF",#"#9933FF",'#66CC99',"#009966","#006633",
             '#99FF33', "#999900", "#CCCC66", "#666633", 
             "#9966CC") ###ex SST xcolor "#99CC99"
b=c('16','0','7','8','19', '11','13','14','2','3','12','10','9')


type<-factor(NG$cluster,levels=b)#, c(0:14),
b=c('16','0','7','8','19', '11','13','14','2','3','12','10','9')


type<-factor(NG$cluster,levels=b)#, c(0:14),

tmp <- data.frame(DC1 = NG_DiffussionMap$DC1,
                  DC2 = NG_DiffussionMap$DC2,
                  DC3=NG_DiffussionMap$DC3,
                  timepoint = NG_c

In [ ]:
tiff("NSCs_neurons_destiny_variable_v2_.tiff", res = 400, units = "in", width = 8, height = 5)

ggplot(tmp, aes(x = DC1, y = DC2, colour =type)) +geom_point() +scale_color_manual(values=ucols_NG)+
  theme_minimal() + 
  theme(axis.text.x = element_text(size=15, face = "bold", colour = "black",angle=45, hjust=1),
         axis.text.y = element_text(size=15, face = "bold", colour = "black"),
         
         axis.text=element_text(size=15,color="black"), 
         axis.title=element_text(size=15,face="bold"),
         text=element_text(face = "bold",family="Arial",size=15),
         legend.text = element_text(size=15, color="black"), legend.title = element_blank())
dev.off()

In [ ]:
library("scatterplot3d")

In [ ]:
colors <- ucols_NG
colors <- colors[as.numeric(tmp$type)]

In [ ]:
tiff("NSCs_neurons_destiny_variable_3D_v4_2.tiff", res = 400, units = "in", width = 6, height = 6)

scatterplot3d(x=tmp$DC1, y=tmp$DC3, z=tmp$DC2, color=colors,pch = 16,angle=55)
legend("right", legend = levels(tmp$type),
      col = ucols_NG , pch = 16)
dev.off()

In [ ]:
saveRDS(NG_DiffussionMap, "NG_DiffussionMap_v4.rds")

In [ ]:
matrix_df<-as.data.frame(matrix)

In [ ]:
#try to plot genes with gradient in the 3D plo
#create a color band per gene 

df_3D <- data.frame(DC1 = eigenvectors(NG_DiffussionMap)[, 1], DC2 = eigenvectors(NG_DiffussionMap)[, 2],
                    DC3 = eigenvectors(NG_DiffussionMap)[, 3],
                 GALNTL6 = matrix_df$GALNTL6, 
                   FRMPD4 = matrix_df$FRMPD4,
                   AC073050.1=matrix_df$AC073050.1)
# Define colour pallete
cols = brewer.pal(9, "RdPu")#YlOrRd rev
#cols = c("#FFFFCC","#FF3333")
pal = colorRampPalette(cols)
df_3D$order = findInterval(df_3D$GALNTL6, sort(df_3D$GALNTL6))
col=pal(nrow(df_3D))[df_3D$order]
tiff("NSCs_neurons_destiny_GALNTL6.tiff", res = 400, units = "in", width = 6, height = 6)

scatterplot3d(x=df_3D$DC1, y=df_3D$DC3, z=df_3D$DC2, color=col,pch = 16,angle=55)
dev.off()

In [ ]:
rr<-random_root(NG_DiffussionMap)
rr

In [ ]:
dpt <- DPT(NG_DiffussionMap,tips = random_root(NG_DiffussionMap))

In [ ]:
df <- data.frame(DC1 = eigenvectors(NG_DiffussionMap)[, 1], DC2 = eigenvectors(NG_DiffussionMap)[, 2], 
                 dptval = dpt$dpt, cluster = type)

In [ ]:
p1 <- ggplot(df) + geom_point(aes(x = DC1, y = DC2, color = dptval))+ scale_color_viridis_c() +
  theme_minimal() + 
  theme(axis.text.x = element_text(size=15, face = "bold", colour = "black",angle=45, hjust=1),
         axis.text.y = element_text(size=15, face = "bold", colour = "black"),
          axis.text=element_text(size=15,color="black"), 
         axis.title=element_text(size=15,face="bold"),
         text=element_text(face = "bold",family="Arial",size=15),
         legend.text = element_text(size=15, color="black"), legend.title = element_blank())
tiff("pseudotime_NSCs_neurons_destiny_variable_v4.tiff", res = 400, units = "in", width = 8, height = 5)
p1
dev.off()

In [ ]:
df_3D <- data.frame(DC1 = eigenvectors(NG_DiffussionMap)[, 1], DC2 = eigenvectors(NG_DiffussionMap)[, 2],
                    DC3 = eigenvectors(NG_DiffussionMap)[, 3],
                 dptval = dpt$dpt, cluster = type)
# Define colour pallete
cols = rev(brewer.pal(9, "YlGnBu"))#YlOrRd
pal = colorRampPalette(cols)
df_3D$order = findInterval(df_3D$dptval, sort(df_3D$dptval))
col=pal(nrow(df_3D))[df_3D$order]
tiff("pseudotime_NSCs_neurons_destiny_variable_3D_blue_v4.tiff", res = 400, units = "in", width = 6, height = 6)

scatterplot3d(x=df_3D$DC1, y=df_3D$DC3, z=df_3D$DC2, color=col,pch = 16,angle=55)

dev.off()

legend_image <- as.raster(matrix(pal(9), ncol=1))
tiff("pseudotime_NSCs_neurons_destiny_variable_3D_legend_blue_v3.tiff", res = 400, units = "in", width = 3, height = 5)

plot(c(0,2),c(0,1),type = 'n', axes = F,xlab = '', ylab = '')

rasterImage(legend_image, 0, 0, 1,1)
dev.off()

In [ ]:
# Define colour pallete
cols = rev(brewer.pal(9, "YlOrRd"))#YlOrRd
pal = colorRampPalette(cols)
df_3D$order = findInterval(df_3D$dptval, sort(df_3D$dptval))
col=pal(nrow(df_3D))[df_3D$order]
tiff("pseudotime_NSCs_neurons_destiny_variable_3D_red_v2.tiff", res = 400, units = "in", width = 6, height = 6)

scatterplot3d(x=df_3D$DC1, y=df_3D$DC3, z=df_3D$DC2, color=col,pch = 16,angle=55)

dev.off()

legend_image <- as.raster(matrix(pal(9), ncol=1))
tiff("pseudotime_NSCs_neurons_destiny_variable_3D_legend_red_v2.tiff", res = 400, units = "in", width = 3, height = 5)

plot(c(0,2),c(0,1),type = 'n', axes = F,xlab = '', ylab = '')

rasterImage(legend_image, 0, 0, 1,1)
dev.off()

In [ ]:
#save dpt values to add to meta data
dpt_df<-data.frame(dpt=dpt$dpt,cluster = type,cells=NG_cells)

data_dpt_NG<-dpt_df["dpt"]
head(data_dpt_NG)

#add pseudotime meta_data to NG
NG <- AddMetaData(
  object = NG,
  metadata = data_dpt_NG,
  col.name = 'data_dpt_NSCs_neurons'
)

tiff("UMAP_psdt_variable_NSCs_neurons_v4.tiff", res = 400, units = "in", width = 5, height = 4)

FeaturePlot(NG, features = "data_dpt_NSCs_neurons", pt.size = 0.3) & scale_color_viridis_c() & theme_void()
dev.off() 

In [ ]:
saveRDS(NG, "NSCs_neurons_CCA_scaled_Dif_components.rds")


In [ ]:
#second part, gene relevance

In [ ]:
NG_DiffussionMap<-readRDS("NG_DiffussionMap_v4.rds")


In [ ]:
future::plan("multicore", workers = 12) # do parallel


In [ ]:
#genes driving the difusion

gr <- gene_relevance(NG_DiffussionMap)

In [ ]:
p_gr<-plot(gr)

In [ ]:
tiff("NSCs_neurons_destiny_variable_relevance_v4.tiff", res = 400, units = "in", width = 6, height = 6)
p_gr
dev.off()

In [ ]:
gr_2 <- gene_relevance(NG_DiffussionMap, dims=1:3)

In [ ]:
p_gr<-plot(gr_2)
p_gr

tiff("NSCs_neurons_destiny_variable_relevance_v4_3D.tiff", res = 400, units = "in", width = 6, height = 6)
p_gr
dev.off()

saveRDS(gr_2, "gr_v4_3D.rds")


In [ ]:
gr_2=readRDS("gr_v4_3D.rds")

In [ ]:
NG_DiffussionMap=readRDS("NG_DiffussionMap_v4.rds")

In [ ]:
d <- plot(gr_2,genesdims = 1:3,)$data
d

In [ ]:
genes<-c("DPP10","KCNIP4","CALM1","DCC","MCTP1","HSPA1A","ROBO2","FRMPD4")
p<-plot(gr_2, genes,faceter = facet_wrap(~Gene,ncol = 4))#,faceter = facet_wrap(~Gene)
p
tiff("gene_Relevance_genes_derivatives_gr3D.tiff", res = 600, units = "in", width = 10, height = 5)
p
dev.off()

In [ ]:
genes<-c("DPP10","KCNIP4","CALM1","DCC","ROBO2","FRMPD4")
p<-plot(gr_2, genes,faceter = facet_wrap(~Gene,ncol = 2))#,faceter = facet_wrap(~Gene)
p
tiff("gene_Relevance_genes_derivatives_gr3D_subset.tiff", res = 600, units = "in", width = 5, height = 6)
p
dev.off()